# 附加练习作业（选做，折平时分）  seq2seq翻译改进练习

# 导入库

In [1]:
#首先执行GPU资源分配代码，勿删除。
import GPU
GPU.show()
GPU.alloc(0,1024)

Sun Jun 25 19:21:54 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.161.03   Driver Version: 470.161.03   CUDA Version: 11.4     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA GeForce ...  Off  | 00000000:02:00.0 Off |                  N/A |
| 41%   45C    P8    33W / 350W |  10247MiB / 24268MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  NVIDIA GeForce ...  Off  | 00000000:03:00.0 Off |                  N/A |
| 65%   

In [2]:
import tensorflow as tf
import re
from tensorflow import keras
from tensorflow.keras.layers import Input,LSTM,Dense,Embedding,GRU
from tensorflow.keras.models import Model,load_model
from tensorflow.keras.utils import plot_model
import pandas as pd
import numpy as np

In [3]:
N_UNITS = 256
BATCH_SIZE = 64
EPOCH = 50
NUM_SAMPLES = 21000
embedding_dim = 256

# 数据处理

In [4]:
data_path = 'data/cmn.txt'

In [5]:
df = pd.read_table(data_path,header=None).iloc[:NUM_SAMPLES,:2,]

In [6]:
df.tail()

,0,1
20995,I listen to the radio after dinner.,晚饭后我听收音机。
20996,I live in an apartment in the city.,我住在城市中的一間公寓裡。
20997,I look forward to hearing from you.,我期待收到你的來信。
20998,I look forward to seeing you again.,我期待著再次見到你。
20999,I lost my ticket. What should I do?,我丢了我的票。我該怎麼辦？


In [7]:
df.columns=['inputs','targets']
df['targets'] = df['targets'].apply(lambda x: '\t'+x+'\n')
df.head()

,inputs,targets
0,Hi.,\t嗨。\n
1,Hi.,\t你好。\n
2,Run.,\t你用跑的。\n
3,Stop!,\t住手！\n
4,Wait!,\t等等！\n


In [8]:
input_texts = df.inputs.values.tolist()
target_texts = df.targets.values.tolist()

In [9]:
# 预处理英文句子
def preprocess_sentence(sentence):
    # 添加空格以分隔符号
    sentence = re.sub(r"([?.!,¿])", r" \1 ", sentence)
    # 将非字母和非标点符号的字符替换为空格
    sentence = re.sub(r"[^a-zA-Z?.!,¿]+", " ", sentence)
    # 去除多余的空格
    sentence = re.sub(r'[" "]+', " ", sentence)
    return sentence.strip()

input_texts=[ preprocess_sentence(t) for t in input_texts]

In [10]:
# 4,加载数据;使用tokenizer做词嵌入;分割数据为train,vaild,test数据集
def max_length(tensor):
    """找到数据集中padding的最大值"""
    return max(len(t) for t in tensor)

def tokenize(lang, china = False):
    """将数据集做padding"""
    lang_tokenizer = tf.keras.preprocessing.text.Tokenizer(filters='', char_level = china)
    lang_tokenizer.fit_on_texts(lang)
    tensor = lang_tokenizer.texts_to_sequences(lang)
    tensor = tf.keras.preprocessing.sequence.pad_sequences(tensor, padding='post')
    return tensor, lang_tokenizer

## 向量化

In [11]:
input_tensor, inp_lang_tokenizer = tokenize(input_texts)

In [12]:
target_tensor, targ_lang_tokenizer = tokenize(target_texts, china=True)
max_length_targ= max_length(target_tensor)
max_length_targ

22

In [13]:
#PADDING字符占用一个码，需要+1
vocab_inp_size = len(inp_lang_tokenizer.word_index)+1
vocab_tar_size = len(targ_lang_tokenizer.word_index)+1

In [14]:
max_length_inp =  max_length(input_tensor)
max_length_inp

12

In [15]:
# 6,测试数据转化结果 
def convert(lang, tensor):
    for t in tensor:
        if t != 0:
            print ("%d ----> %s" % (t, lang.index_word[t]))
            
print("Input Language; index to word mapping")
convert(inp_lang_tokenizer, input_tensor[-1])

Input Language; index to word mapping
2 ----> i
174 ----> lost
14 ----> my
670 ----> ticket
1 ----> .
23 ----> what
79 ----> should
2 ----> i
15 ----> do
4 ----> ?


In [16]:
print("Target Language; index to word mapping")
convert(targ_lang_tokenizer, target_tensor[-1])

Target Language; index to word mapping
1 ----> 	
4 ----> 我
673 ----> 丢
7 ----> 了
4 ----> 我
5 ----> 的
437 ----> 票
3 ----> 。
4 ----> 我
172 ----> 該
93 ----> 怎
43 ----> 麼
648 ----> 辦
9 ----> ？
2 ----> 



In [17]:
from sklearn.model_selection import train_test_split
# 5,拆分训练集和验证集
# Creating training and validation sets using an 80-20 split
input_tensor_train, input_tensor_val, target_tensor_train, target_tensor_val = train_test_split(input_tensor, target_tensor, test_size=1000)

# 打印数据集长度 - Show length
print(len(input_tensor_train), len(target_tensor_train), len(input_tensor_val), len(target_tensor_val))

20000 20000 1000 1000


In [18]:
#decoder的label tensor
output_tensor_train=np.zeros_like(target_tensor_train)
output_tensor_train[:,:-1]=target_tensor_train[:,1:]

output_tensor_val=np.zeros_like(target_tensor_val)
output_tensor_val[:,:-1]=target_tensor_val[:,1:]

In [19]:
input_tensor.shape, target_tensor.shape

((21000, 12), (21000, 22))

# 创建模型（作业要点）

In [20]:
# def create_model(n_input,n_output,n_units):
#     #训练阶段
#     #encoder
#     encoder_input = Input(shape = (None, ))
#     embeddin = keras.layers.Embedding(n_input, embedding_dim)
#     #encoder输入维度n_input为每个时间步的输入xt的维度，这里是用来one-hot的英文字符数
#     encoder = LSTM(n_units, return_state=True)
#     #n_units为LSTM单元中每个门的神经元的个数，return_state设为True时才会返回最后时刻的状态h,c
#     _,encoder_h,encoder_c = encoder(embeddin(encoder_input))
#     encoder_state = [encoder_h,encoder_c]
#     #保留下来encoder的末状态作为decoder的初始状态
    
#     #decoder
#     decoder_input = Input(shape = (None, ))
#     embeddout = keras.layers.Embedding(n_output, embedding_dim)
#     #decoder的输入维度为中文字符数
#     decoder = LSTM(n_units,return_sequences=True, return_state=True)
#     #训练模型时需要decoder的输出序列来与结果对比优化，故return_sequences也要设为True
#     decoder_output, _, _ = decoder(embeddout(decoder_input),initial_state=encoder_state)
#     #在训练阶段只需要用到decoder的输出序列，不需要用最终状态h.c
#     decoder_dense = Dense(n_output,activation='softmax')
#     decoder_output = decoder_dense(decoder_output)
#     #输出序列经过全连接层得到结果
    
#     #生成的训练模型
#     model = Model([encoder_input,decoder_input],decoder_output)
#     #第一个参数为训练模型的输入，包含了encoder和decoder的输入，第二个参数为模型的输出，包含了decoder的输出
    
#     #推理阶段，用于预测过程
#     #推断模型—encoder
#     encoder_infer = Model(encoder_input,encoder_state)
    
#     #推断模型-decoder
#     decoder_state_input_h = Input(shape=(n_units,))
#     decoder_state_input_c = Input(shape=(n_units,))    
#     decoder_state_input = [decoder_state_input_h, decoder_state_input_c]#上个时刻的状态h,c   
    
#     decoder_infer_output, decoder_infer_state_h, decoder_infer_state_c = decoder(embeddout(decoder_input),initial_state=decoder_state_input)
#     decoder_infer_state = [decoder_infer_state_h, decoder_infer_state_c]#当前时刻得到的状态
#     decoder_infer_output = decoder_dense(decoder_infer_output)#当前时刻的输出
#     decoder_infer = Model([decoder_input]+decoder_state_input,[decoder_infer_output]+decoder_infer_state)
    
#     return model, encoder_infer, decoder_infer

In [21]:
# def create_model(n_input, n_output, n_units):
#     # 训练阶段
#     # Encoder
#     encoder_input = Input(shape=(None,))
#     embedding_dim = 64  # 假设embedding维度为64
#     embedding = Embedding(n_input, embedding_dim)
#     encoder = GRU(n_units, return_sequences=False, return_state=True)  # 使用GRU替换LSTM
#     encoder_output, encoder_state = encoder(embedding(encoder_input))
    
#     # Decoder
#     decoder_input = Input(shape=(None,))
#     decoder_embedding = Embedding(n_output, embedding_dim)
#     decoder = GRU(n_units, return_sequences=True, return_state=True)  # 使用GRU替换LSTM
#     decoder_output, _ = decoder(decoder_embedding(decoder_input), initial_state=encoder_state)
    
#     decoder_dense = Dense(n_output, activation='softmax')
#     decoder_output = decoder_dense(decoder_output)
    
#     # 生成训练模型
#     model = Model([encoder_input, decoder_input], decoder_output)
    
#     # 推理阶段，用于预测过程
#     # 推断模型 - Encoder
#     encoder_infer = Model(encoder_input, encoder_state)
    
#     # 推断模型 - Decoder
#     decoder_state_input = Input(shape=(n_units,))
#     decoder_infer_output, decoder_infer_state = decoder(decoder_embedding(decoder_input), initial_state=decoder_state_input)
#     decoder_infer_output = decoder_dense(decoder_infer_output)
    
#     decoder_infer = Model([decoder_input, decoder_state_input], [decoder_infer_output, decoder_infer_state])
    
#     return model, encoder_infer, decoder_infer

In [20]:
def create_model(n_input, n_output, n_units):
    # 训练阶段
    # Encoder
    encoder_input = Input(shape=(None,))
    embedding_dim = 256
    embedding = Embedding(n_input, embedding_dim)
    encoder = GRU(n_units, return_sequences=False, return_state=True)
    encoder_output, encoder_state = encoder(embedding(encoder_input))
    
    # Decoder
    decoder_input = Input(shape=(None,))
    decoder_embedding = Embedding(n_output, embedding_dim)
    decoder = GRU(n_units, return_sequences=True, return_state=True)
    decoder_output, _ = decoder(decoder_embedding(decoder_input), initial_state=encoder_state)
    
    decoder_dense = Dense(n_output, activation='softmax')
    decoder_output = decoder_dense(decoder_output)
    
    # 生成训练模型
    model = Model([encoder_input, decoder_input], decoder_output)
    
    # 推理阶段，用于预测过程
    # 推断模型 - Encoder
    encoder_infer = Model(encoder_input, encoder_state)
    
    # 推断模型 - Decoder
    decoder_state_input = Input(shape=(n_units,))
    decoder_infer_output, decoder_infer_state = decoder(decoder_embedding(decoder_input), initial_state=decoder_state_input)
    decoder_infer_output = decoder_dense(decoder_infer_output)
    
    decoder_infer = Model([decoder_input, decoder_state_input], [decoder_infer_output, decoder_infer_state])
    
    return model, encoder_infer, decoder_infer

In [21]:
model_train, encoder_infer, decoder_infer = create_model(vocab_inp_size, vocab_tar_size, N_UNITS)

In [22]:
#input_tensor, target_tensor

model_train.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')

In [23]:
encoder_infer.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, None)]            0         
                                                                 
 embedding (Embedding)       (None, None, 256)         1299200   
                                                                 
 gru (GRU)                   [(None, 256),             394752    
                              (None, 256)]                       
                                                                 
Total params: 1,693,952
Trainable params: 1,693,952
Non-trainable params: 0
_________________________________________________________________


# 模型训练

In [24]:
model_train.fit([input_tensor_train, target_tensor_train],output_tensor_train,batch_size=BATCH_SIZE,epochs=EPOCH,validation_data=([input_tensor_val, target_tensor_val],output_tensor_val) )

Epoch 1/50
313/313 [==============================] - 11s 16ms/step - loss: 2.1512 - val_loss: 1.8062
Epoch 2/50
313/313 [==============================] - 4s 13ms/step - loss: 1.6675 - val_loss: 1.5900
Epoch 3/50
313/313 [==============================] - 4s 14ms/step - loss: 1.4885 - val_loss: 1.4772
Epoch 4/50
313/313 [==============================] - 4s 14ms/step - loss: 1.3702 - val_loss: 1.3932
Epoch 5/50
313/313 [==============================] - 4s 14ms/step - loss: 1.2766 - val_loss: 1.3362
Epoch 6/50
313/313 [==============================] - 4s 13ms/step - loss: 1.1961 - val_loss: 1.2876
Epoch 7/50
313/313 [==============================] - 4s 13ms/step - loss: 1.1233 - val_loss: 1.2408
Epoch 8/50
313/313 [==============================] - 4s 14ms/step - loss: 1.0572 - val_loss: 1.2080
Epoch 9/50
313/313 [==============================] - 4s 13ms/step - loss: 0.9963 - val_loss: 1.1819
Epoch 10/50
313/313 [==============================] - 4s 13ms/step - loss: 0.9411 - val_l

# 预测序列

In [25]:
def predict_chinese(source,encoder_inference, decoder_inference, n_steps, features):
    #先通过推理encoder获得预测输入序列的隐状态
    state = encoder_inference.predict(source)
    #第一个字符'\t',为起始标志
    predict_seq = np.zeros((1,1))
    predict_seq[0,0]=1   ##target_dict['\t']

    output = ''
    #开始对encoder获得的隐状态进行推理
    #每次循环用上次预测的字符作为输入来预测下一次的字符，直到预测出了终止符
    for i in range(n_steps):#n_steps为句子最大长度
        #给decoder输入上一个时刻的h,c隐状态，以及上一次的预测字符predict_seq
        yhat,h = decoder_inference.predict([predict_seq]+[state])
        #注意，这里的yhat为Dense之后输出的结果，因此与h不同
        char_index = np.argmax(yhat[0,-1,:])
        char = targ_lang_tokenizer.index_word[char_index]
        output += char
        state = h#本次状态做为下一次的初始状态继续传递
        predict_seq = np.zeros((1,1))
        predict_seq[0,0]=char_index
        if char == '\n':#预测到了终止符则停下来
            break
    return output

In [26]:
def convertinp(tensor):
    s=''
    for t in tensor:
        if t != 0:
            s += inp_lang_tokenizer.index_word[t]+' '
    return s

In [27]:
for i in range(20):
    test = input_tensor_val[i:i+1,:]#i:i+1保持数组是三维
    out = predict_chinese(test,encoder_infer,decoder_infer,max_length_targ,vocab_tar_size)

    print(convertinp(input_tensor_val[i]))
    print(out)

you are going too far . 
你的太多了第一起。

my grandfather lived a long life . 
我的祖父是活费了。

whose is this car ? 
這把傘是誰嗎?

i can t resist sweet things . 
我不能有一些錢。

it wasn t interesting at all . 
它根本不是很有用。

i guess it depends on the weather . 
我想知道這個派對。

the stripes were horizontal . 
這個小男孩把所有把書被開掉。

my father will kill me . 
我爸会杀了我的。

turn off the tv . 
把電視關了。

lie on your right side . 
警方不應該收取。

he is very fond of music . 
他非常喜欢音乐。

i was very tired last night . 
昨晚我感到有很多以前打了。

we have many goals . 
我們有用他的。

i am too short . 
我是對的。

i guess that would be all right . 
我猜那應該可以的。

my passport was stolen . 
我护照给偷了。

one plus two equals three . 
有三个姐姐。

we know that . 
我们知道。

this tie doesn t go with my suit . 
这个男孩，我不在乎是否新會回來。

sorry , i don t have any change . 
不好意思，我没能做过。

